In [4]:
# Data handling libraries
import pandas as pd
pd.options.display.max_rows = 10000
import numpy as np
import boto3
import io
import requests as req

# Initialize S3 client, location of files for this project
s3_client = boto3.client('s3')
s3_resource = boto3.resource('s3')

s3_bucket = "wri-public-data"

WB_DATA = "resourcewatch/world_bank_data_long_and_wide/"
CONVERSIONS = "resourcewatch/blog_data/GHG-GDP_Divergence_D3/Conversions/"

# Functions for reading and uploading data to/from S3
def read_from_S3(bucket, key, index_col=0):
    obj = s3_client.get_object(Bucket=bucket, Key=key)
    df = pd.read_csv(io.BytesIO(obj['Body'].read()), index_col=[index_col], encoding="utf8")
    return(df)

def write_to_S3(df, bucket, key):
    csv_buffer = io.StringIO()
    df.to_csv(csv_buffer)
    s3_resource.Object(bucket, key).put(Body=csv_buffer.getvalue())

In [2]:
## World Bank data series codes and names

data_names_and_codes = {'EG.ELC.ACCS.ZS': 'Access to electricity (% of population)',
 'EG.FEC.RNEW.ZS': 'Renewable energy consumption (% of total final energy consumption)',
 'IT.NET.USER.ZS': 'Individuals using the Internet (% of population)',
 'NE.CON.PRVT.PC.KD': 'Household final consumption expenditure per capita (constant 2010 US$)',
 'NV.IND.TOTL.KD': 'Industry, value added (constant 2010 US$)',
 'NY.GDP.TOTL.RT.ZS': 'Total natural resources rents (% of GDP)',
 'SG.GEN.PARL.ZS': 'Proportion of seats held by women in national parliaments (%)',
 'SL.EMP.TOTL.SP.ZS': 'Employment to population ratio, 15+, total (%) (modeled ILO estimate)',
 'SM.POP.NETM': 'Net migration',
 'SP.DYN.LE00.IN': 'Life expectancy at birth, total (years)',
 'SP.URB.TOTL.IN.ZS': 'Urban population (% of total)',
 'TM.VAL.MRCH.CD.WT': 'Merchandise imports (current US$)',
 'NY.GDP.MKTP.CD': 'GDP (current US$)'}

In [17]:
# Load conversions from wb_name to iso3
wb_name_to_iso3_conversion = read_from_S3(s3_bucket, CONVERSIONS+"World Bank to ISO3 name conversion.csv")

# Provide function to map from wb_name to ISO3
def add_iso(name):
    try:
        return(wb_name_to_iso3_conversion.loc[name,"ISO"])
    except:
        return(np.nan)

In [24]:
wb_name_to_iso3_conversion

,ISO
Afghanistan,AFG
Albania,ALB
Algeria,DZA
American Samoa,ASM
Andorra,AND
Angola,AGO
Anguila,AIA
Antigua and Barbuda,ATG
Argentina,ARG
Armenia,ARM


## Data in long form

In [21]:
indicators = list(data_names_and_codes.keys())
seed = indicators[0]
print(seed)
res = req.get("http://api.worldbank.org/countries/all/indicators/{}?format=json&per_page=10000".format(seed))
#print(res.text)
data = pd.io.json.json_normalize(res.json()[1])
data = data[["country.value", "date", "value"]]
value_name = data_names_and_codes[seed]
data.columns = ["Country Name", "Year", value_name]
data = data.set_index(["Country Name", "Year"])
all_world_bank_data = data

for indicator in indicators[1:]:
    # Results are paginated
    print(indicator)
    res = req.get("http://api.worldbank.org/countries/all/indicators/{}?format=json&per_page=10000".format(indicator))
    #print(res.text)
    data = pd.io.json.json_normalize(res.json()[1])
    data = data[["country.value", "date", "value"]]
    value_name = data_names_and_codes[indicator]
    data.columns = ["Country Name", "Year", value_name]
    data = data.set_index(["Country Name", "Year"])
    all_world_bank_data = all_world_bank_data.join(data, how="outer")
    
all_world_bank_data = all_world_bank_data.reset_index()
all_world_bank_data["ISO3"] = list(map(add_iso, all_world_bank_data["Country Name"]))
all_world_bank_data = all_world_bank_data.loc[pd.notnull(all_world_bank_data["ISO3"])]
all_world_bank_data = all_world_bank_data.set_index(["Country Name", "Year"])

EG.ELC.ACCS.ZS
EG.FEC.RNEW.ZS
IT.NET.USER.ZS
NE.CON.PRVT.PC.KD
NV.IND.TOTL.KD
NY.GDP.TOTL.RT.ZS
SG.GEN.PARL.ZS
SL.EMP.TOTL.SP.ZS
SM.POP.NETM
SP.DYN.LE00.IN
SP.URB.TOTL.IN.ZS
TM.VAL.MRCH.CD.WT
NY.GDP.MKTP.CD


In [23]:
all_world_bank_data.head()

Access to electricity (% of population)  \
Country Name   Year                                           
European Union 2016                                    None   
               2015                                    None   
               2014                                     100   
               2013                                     100   
               2012                                     100   

                    Renewable energy consumption (% of total final energy consumption)  \
Country Name   Year                                                                      
European Union 2016                                               None                   
               2015                                               None                   
               2014                                    16.053026214829                   
               2013                                   15.1442661475667                   
               2012                                   14.3811492809133                   

                    Individuals using the Internet (% of population)  \
Country Name   Year                                                    
European Union 2016                                 80.7691050337866   
               2015                                 78.4329000006293   
               2014                                 77.1139914118683   
               2013                                 75.5428368269042   
               2012                                 73.7061519154293   

                    Household final consumption expenditure per capita (constant 2010 US$)  \
Country Name   Year                                                                          
European Union 2016                                   19968.1369826023                       
               2015                                   19583.8758139882                       
               2014                                   19245.8917008979                       
               2013                                   19082.6827482441                       
               2012                                   19160.1520852802                       

                    Industry, value added (constant 2010 US$)  \
Country Name   Year                                             
European Union 2016                          3988821725434.18   
               2015                          3933732615792.92   
               2014                          3807777737722.81   
               2013                          3731658584714.89   
               2012                          3776290013765.67   

                    Total natural resources rents (% of GDP)  \
Country Name   Year                                            
European Union 2016                                     None   
               2015                        0.209417246513485   
               2014                        0.319622359791876   
               2013                        0.373596153789426   
               2012                        0.441742736714659   

                    Proportion of seats held by women in national parliaments (%)  \
Country Name   Year                                                                 
European Union 2016                                    28.418549063461              
               2015                                   28.3813206756945              
               2014                                   27.7298367973218              
               2013                                   27.0906547133931              
               2012                                   25.7243875884594              

                    Employment to population ratio, 15+, total (%) (modeled ILO estimate)  \
Country Name   Year                                                                         
European Union 2016                                   52.3869840451532                      
   

In [41]:
for code, name in data_names_and_codes.items():
    long_form = all_world_bank_data[name]
    long_form = long_form.reset_index()
    long_form = long_form[pd.notnull(long_form[name])]
    write_to_S3(long_form, s3_bucket, WB_DATA + "wb_data_long_{}.csv".format(name.replace(" ", "_")))

## Data in wide form

In [49]:
df = all_world_bank_data.reset_index()
index_by_countryName = df.columns[0]
columns_by_year = df.columns[1]

# Only go up to -1 b/c we don't need to do this for the ISO column
names = df.columns[2:-1]

for name in names:
    wide_form = df.pivot(index=index_by_countryName,
           columns=columns_by_year,
           values=name)
    wide_form = wide_form.dropna(how="all", axis=1)
    wide_form = wide_form.reset_index()
    wide_form["ISO3"] = list(map(add_iso, wide_form["Country Name"]))
    wide_form = wide_form[pd.notnull(wide_form["ISO3"])]
    # Carto doesn't except column names that are numbers! 
    wide_form.columns = ["yr_" + str(col) for col in wide_form.columns]
    write_to_S3(wide_form, s3_bucket, WB_DATA + "wb_data_wide_{}.csv".format(name.replace(" ", "_")))

In [5]:
# Read in wide form data

names = list(data_names_and_codes.values())
data = {}

for name in names:
    data[name] = read_from_S3(s3_bucket, WB_DATA + "wb_data_wide_{}.csv".format(name.replace(" ", "_")))
    
df = data[names[0]]
df

,yr_Country Name,yr_1990,yr_1991,yr_1992,yr_1993,yr_1994,yr_1995,yr_1996,yr_1997,yr_1998,...,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,yr_2011,yr_2012,yr_2013,yr_2014,yr_ISO3
0,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27.506411,34.290512,42.400000,47.888466,42.700000,61.514420,69.100000,75.154373,89.500000,AFG
1,Albania,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,ALB
2,Algeria,92.990051,93.393257,93.796013,94.196053,94.590691,94.977211,95.352905,95.715065,96.060997,...,98.490738,98.806519,99.300000,99.443893,99.711174,99.889542,99.973083,99.996918,100.000000,DZA
3,American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ASM
4,Andorra,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,AND
5,Angola,47.833630,47.261116,46.688148,46.112480,45.531395,44.942196,44.342175,43.728622,43.098835,...,37.714977,37.500000,36.405998,35.765507,35.132019,34.600000,33.879036,33.256046,32.000000,AGO
6,Antigua and Barbuda,81.763031,82.435524,83.107567,83.776909,84.440834,85.096649,85.741631,86.373093,86.988312,...,91.564537,92.149811,92.745583,93.350098,93.961617,94.552014,95.198654,95.820679,96.442986,ATG
7,Argentina,90.759087,91.224731,91.689919,92.152405,92.609482,93.058434,93.496567,93.921173,94.329544,...,97.250938,97.629356,98.024818,98.421188,98.820000,99.220192,99.573059,99.831543,100.000000,ARG
8,Armenia,97.251823,97.452187,97.652107,97.849319,98.035034,98.216690,98.387535,98.545059,98.687431,...,99.507156,99.619987,99.739517,99.852890,99.800000,99.983543,99.997940,100.000000,100.000000,ARM
9,Aruba,88.961044,89.251343,89.541191,89.828339,90.110069,90.383690,90.646477,90.895744,91.128769,...,92.647430,92.850510,93.064087,93.286415,93.356292,93.750313,93.988388,94.228210,94.468323,ABW


In [6]:
# Convert back to long form
# https://hackernoon.com/reshaping-data-in-python-fa27dda2ff77
pd.melt(df, id_vars = "yr_Country Name", value_vars = df.columns[1:-1])

,yr_Country Name,variable,value
0,Afghanistan,yr_1990,NaN
1,Albania,yr_1990,100.000000
2,Algeria,yr_1990,92.990051
3,American Samoa,yr_1990,NaN
4,Andorra,yr_1990,100.000000
5,Angola,yr_1990,47.833630
6,Antigua and Barbuda,yr_1990,81.763031
7,Argentina,yr_1990,90.759087
8,Armenia,yr_1990,97.251823
9,Aruba,yr_1990,88.961044


In [7]:
# Base URL for getting dataset metadata from RW API
# Metadata = Data that describes Data 
url = "https://api.resourcewatch.org/v1/dataset?sort=slug,-provider,userId&status=saved&includes=metadata,vocabulary,widget,layer"

# page[size] tells the API the maximum number of results to send back
# There are currently between 200 and 300 datasets on the RW API
payload = { "application":"rw", "page[size]": 1000}

# Request all datasets, and extract the data from the response
res = req.get(url, params=payload)
data = res.json()["data"]

#############################################################

### Convert the json object returned by the API into a pandas DataFrame
# Another option: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html
datasets_on_api = {}
for ix, dset in enumerate(data):
    atts = dset["attributes"]
    metadata = atts["metadata"]
    layers = atts["layer"]
    widgets = atts["widget"]
    tags = atts["vocabulary"]
    datasets_on_api[atts["name"]] = {
        "rw_id":dset["id"],
        "table_name":atts["tableName"],
        "provider":atts["provider"],
        "date_updated":atts["updatedAt"],
        "num_metadata":len(metadata),
        "metadata": metadata,
        "num_layers":len(layers),
        "layers": layers,
        "num_widgets":len(widgets),
        "widgets": widgets,
        "num_tags":len(tags),
        "tags":tags
    }

# Create the DataFrame, name the index, and sort by date_updated
# More recently updated datasets at the top
current_datasets_on_api = pd.DataFrame.from_dict(datasets_on_api, orient='index')
current_datasets_on_api.index.rename("Dataset", inplace=True)
current_datasets_on_api.sort_values(by=["date_updated"], inplace=True, ascending = False)

In [8]:
# Choose only datasets stored on:
## cartodb, csv, gee, featureservice, bigquery, wms, json, rasdaman
provider = "cartodb"
carto_ids = (current_datasets_on_api["provider"]==provider)
carto_data = current_datasets_on_api.loc[carto_ids]

print("Number of Carto datasets: ", carto_data.shape[0])

Number of Carto datasets:  213


In [32]:
datasets = [ds for ds in carto_data.index if 'greenhouse gas' in ds.lower()]
datasets

['Cli.008 National Greenhouse Gas Emissions-dash',
 'cli.007 Greenhouse Gas Emissions Projections',
 'cli.008 Greenhouse Gas Emissions by Country and Economic Sector']

In [47]:
carto_data.loc[ 'cli.007 Greenhouse Gas Emissions Projections']

rw_id                        3d2ce960-abda-4c9c-bd29-1929e9ca24c9
table_name           cli_007_greenhouse_gas_emissions_projections
provider                                                  cartodb
date_updated                             2018-01-15T17:18:04.231Z
num_metadata                                                    2
metadata        [{'id': '59dd225126cb7a0013147aa7', 'type': 'm...
num_layers                                                      1
layers          [{'id': '1d065607-2b13-40cf-973a-6f55b4de573b'...
num_widgets                                                     1
widgets         [{'id': 'b9192492-20f3-4725-8e90-8795f2176329'...
num_tags                                                        1
tags            [{'type': 'vocabulary', 'attributes': {'resour...
Name: cli.007 Greenhouse Gas Emissions Projections, dtype: object

In [15]:
# Template query string used to query RW datasets
query_base = "https://api.resourcewatch.org/v1/query/{}?sql={}"

# Template SQL string used in RW query
sql = "SELECT * FROM {}"

In [37]:
dataset = 'cit.029 Municipal Waste'
table_name = carto_data.loc[dataset, "table_name"]
rw_id = carto_data.loc[dataset, "rw_id"]
query_sql = sql.format(table_name)
query = query_base.format(rw_id, query_sql)    

res = req.get(query)
data = res.json()["data"]
data[0].keys()

dict_keys(['cartodb_id', 'the_geom', 'the_geom_webmercator', 'country', 'yr_1990', 'yr_1991', 'yr_1992', 'yr_1993', 'yr_1994', 'yr_1995', 'yr_1996', 'yr_1997', 'yr_1998', 'yr_1999', 'yr_2000', 'yr_2001', 'yr_2002', 'yr_2003', 'yr_2004', 'yr_2005', 'yr_2006', 'yr_2007', 'yr_2008', 'yr_2009', 'yr_2010', 'yr_2011', 'yr_2012', 'yr_2013', 'yr_2014', 'yr_2015', '_2012_2015_avg'])

In [38]:
df = pd.DataFrame(data)
df.head()

,_2012_2015_avg,cartodb_id,country,the_geom,the_geom_webmercator,yr_1990,yr_1991,yr_1992,yr_1993,yr_1994,...,yr_2006,yr_2007,yr_2008,yr_2009,yr_2010,yr_2011,yr_2012,yr_2013,yr_2014,yr_2015
0,522.890000,13,Iceland,0106000020E6100000050000000103000000010000000F...,0106000020110F0000050000000103000000010000000F...,NaN,NaN,NaN,NaN,NaN,...,561.905,558.790,650.705,412.862,481.144,495.295,513.820,517.897,536.953,NaN
1,587.203000,14,Ireland,0106000020E6100000070000000103000000010000000E...,0106000020110F0000070000000103000000010000000E...,NaN,NaN,NaN,NaN,NaN,...,799.594,776.457,718.871,651.383,624.863,617.076,587.203,NaN,NaN,NaN
2,448.354667,26,Portugal,0106000020E61000001100000001030000000100000005...,0106000020110F00001100000001030000000100000005...,300.504,NaN,328.561,357.563,380.323,...,465.496,471.146,518.256,520.074,516.134,490.433,453.257,439.688,452.119,NaN
3,314.840750,27,Slovak Republic,0106000020E6100000010000000103000000010000003D...,0106000020110F0000010000000103000000010000003D...,NaN,NaN,301.602,300.452,299.271,...,283.636,293.800,313.341,307.134,318.842,311.004,306.343,303.666,319.978,329.376
4,414.217000,28,Slovenia,0106000020E61000000100000001030000000100000007...,0106000020110F00000100000001030000000100000007...,NaN,NaN,NaN,NaN,NaN,...,516.038,524.951,541.256,523.592,489.884,415.163,361.826,414.445,432.528,448.069


In [ ]:
### Insert - try to do the join to shapefiles, see how many don't match ISOs, add new country names when necessary
## Report - years correctly matched for each data set, validate we matched right years/range, and # of countries matched


## Iterate
## Put up new tables first - check everything, when it's good, rename old tables


In [39]:
pd.melt(df, id_vars = "country", value_vars = ['_2012_2015_avg'] + list(df.columns[5:]))

,country,variable,value
0,Iceland,_2012_2015_avg,522.890000
1,Ireland,_2012_2015_avg,587.203000
2,Portugal,_2012_2015_avg,448.354667
3,Slovak Republic,_2012_2015_avg,314.840750
4,Slovenia,_2012_2015_avg,414.217000
5,Spain,_2012_2015_avg,451.394500
6,Sweden,_2012_2015_avg,446.197000
7,Switzerland,_2012_2015_avg,724.606500
8,Turkey,_2012_2015_avg,406.146000
9,United Kingdom,_2012_2015_avg,487.646000


In [42]:
dataset = 'cli.008 Greenhouse Gas Emissions by Country and Economic Sector'
table_name = carto_data.loc[dataset, "table_name"]
rw_id = carto_data.loc[dataset, "rw_id"]
query_sql = sql.format(table_name)
query = query_base.format(rw_id, query_sql)    

res = req.get(query)
data = res.json()["data"]
data[0].keys()

dict_keys(['cartodb_id', 'the_geom', 'the_geom_webmercator', 'country', 'year', 'total_ghg_emissions_excluding_land_use_change_and_forestry_mtc', 'total_ghg_emissions_including_land_use_change_and_forestry_mtc', 'total_co2_excluding_land_use_change_and_forestry_mtco2', 'total_ch4_mtco2e', 'total_n2o_mtco2e', 'total_f_gas_mtco2e', 'total_co2_including_land_use_change_and_forestry_mtco2', 'total_ch4_including_land_use_change_and_forestry_mtco2e', 'total_n2o_including_land_use_change_and_forestry_mtco2e', 'energy_mtco2e', 'industrial_processes_mtco2e', 'agriculture_mtco2e', 'waste_mtco2e', 'land_use_change_and_forestry_mtco2', 'bunker_fuels_mtco2', 'electricity_heat_mtco2', 'manufacturing_construction_mtco2', 'transportation_mtco2', 'other_fuel_combustion_mtco2e', 'fugitive_emissions_mtco2e', 'cartodb_georef_status'])

In [51]:
df = pd.DataFrame(data)
df.head()

,agriculture_mtco2e,bunker_fuels_mtco2,cartodb_georef_status,cartodb_id,country,electricity_heat_mtco2,energy_mtco2e,fugitive_emissions_mtco2e,industrial_processes_mtco2e,land_use_change_and_forestry_mtco2,...,total_co2_excluding_land_use_change_and_forestry_mtco2,total_co2_including_land_use_change_and_forestry_mtco2,total_f_gas_mtco2e,total_ghg_emissions_excluding_land_use_change_and_forestry_mtc,total_ghg_emissions_including_land_use_change_and_forestry_mtc,total_n2o_including_land_use_change_and_forestry_mtco2e,total_n2o_mtco2e,transportation_mtco2,waste_mtco2e,year
0,14.794115,NaN,True,1,Afghanistan,NaN,9.769908,0.020004,0.377208,0.000000,...,9.801200,9.801200,0.325912,33.366148,33.366148,5.488617,5.488617,NaN,8.424918,2014
1,1.809676,0.13,True,39,Congo,0.46,4.361613,1.370927,0.240961,12.294671,...,4.129085,15.427756,0.010129,6.998262,19.292932,1.304915,1.111915,2.02,0.586012,2014
2,8.096849,NaN,True,94,Laos,NaN,1.746658,0.108251,1.204047,18.066333,...,1.949248,18.148581,0.009583,11.551705,29.618038,3.104351,2.539351,NaN,0.504151,2014
3,0.425628,NaN,True,186,Vanuatu,NaN,0.177401,0.000000,0.007085,-0.014007,...,0.153888,0.139881,0.007085,0.729092,0.715085,0.169212,0.169212,NaN,0.118978,2014
4,36.052821,4.89,True,187,Venezuela,55.09,215.666287,60.013718,11.347680,66.399625,...,188.408619,253.324244,6.915974,271.135416,337.535041,14.164957,13.828957,50.12,8.068627,2014


In [49]:
import datetime

res = pd.melt(df, id_vars = ["country", "year"], value_vars = ['total_ghg_emissions_excluding_land_use_change_and_forestry_mtc', 'total_ghg_emissions_including_land_use_change_and_forestry_mtc', 'total_co2_excluding_land_use_change_and_forestry_mtco2', 'total_ch4_mtco2e', 'total_n2o_mtco2e', 'total_f_gas_mtco2e', 'total_co2_including_land_use_change_and_forestry_mtco2', 'total_ch4_including_land_use_change_and_forestry_mtco2e', 'total_n2o_including_land_use_change_and_forestry_mtco2e', 'energy_mtco2e', 'industrial_processes_mtco2e', 'agriculture_mtco2e', 'waste_mtco2e', 'land_use_change_and_forestry_mtco2', 'bunker_fuels_mtco2', 'electricity_heat_mtco2', 'manufacturing_construction_mtco2', 'transportation_mtco2', 'other_fuel_combustion_mtco2e', 'fugitive_emissions_mtco2e'])
res['year'] = [datetime.datetime(year=yr, month=1, day=2) for yr in res['year']]
res

,country,year,variable,value
0,Afghanistan,2014-01-02,total_ghg_emissions_excluding_land_use_change_...,3.336615e+01
1,Congo,2014-01-02,total_ghg_emissions_excluding_land_use_change_...,6.998262e+00
2,Laos,2014-01-02,total_ghg_emissions_excluding_land_use_change_...,1.155171e+01
3,Vanuatu,2014-01-02,total_ghg_emissions_excluding_land_use_change_...,7.290919e-01
4,Venezuela,2014-01-02,total_ghg_emissions_excluding_land_use_change_...,2.711354e+02
5,Vietnam,2014-01-02,total_ghg_emissions_excluding_land_use_change_...,2.702983e+02
6,Zambia,2014-01-02,total_ghg_emissions_excluding_land_use_change_...,5.120099e+01
7,Zimbabwe,2014-01-02,total_ghg_emissions_excluding_land_use_change_...,2.772440e+01
8,Austria,2014-01-02,total_ghg_emissions_excluding_land_use_change_...,7.514770e+01
9,Belarus,2014-01-02,total_ghg_emissions_excluding_land_use_change_...,8.958018e+01


## World Bank country names to delete

Arab World, Middle income, Europe & Central Asia (IDA & IBRD countries), IDA total, Latin America & the Caribbean (IDA & IBRD countries), Middle East & North Africa (IDA & IBRD countries), *blank* (ID 268), Europe & Central Asia (excluding high income), IBRD only, IDA only, Early-demographic dividend, Latin America & the Caribbean (excluding high income), Middle East & North Africa, Middle East & North Africa (excluding high income), Late-demographic dividend, Pacific island small states, Europe & Central Asia, European Union, High income, IDA & IBRD total, IDA blend, Caribbean small states, Central Europe and the Baltics, East Asia & Pacific, East Asia & Pacific  (excluding high income), Low & middle income, Lower middle income, Other small states, Latin America & Caribbean, East Asia & Pacific (IDA & IBRD countries), Euro area, OECD members, North America, Middle East & North Africa (excluding high income),  Post-demographic dividend, Small states, South Asia, Upper middle income, World, heavily indebted poor countries (HIPC), Least developed countries: UN classification, *blank* (ID 267), *blank* (ID 265), Latin America & Caribbean, IDA & IBRD total, IBRD only, Europe & Central Asia, sub-Saharan Africa (excluding high income), Macao SAR China,  sub-Saharan Africa, pre-demographic dividend, South Asia (IDA & IBRD), sub-Saharan Africa (IDA & IBRD), Upper middle income, fragile and conflict affected 